In [1]:
import csv
from collections import OrderedDict
import tqdm

In [2]:
from __future__ import division #use this if on python 2.7. It will work without, but it's actually pretty important, so... 

In [5]:

#"""
with open("train.csv") as training_csv:
    train_list_tmp = csv.reader(training_csv)
    train_list = []
    for row in train_list_tmp:
        train_list.append(row)
    train_list = train_list[1:]#"""
with open("test.csv") as test_csv:
    test_list_tmp = csv.reader(test_csv)
    test_list = []
    for row in test_list_tmp:
        test_list.append(row)
    test_list = test_list[1:]

In [41]:
def generateData(list_id, first_column_identifier):
    data = []
    for row in tqdm.tqdm(list_id):
        selectSequence = []
        isInSelectSequence = False
        currentSelectSequence = 0
        hotkeySequence = []
        isInHotkeySequence = False
        currentHotkeySequence = 0
        features = OrderedDict()
        features[first_column_identifier] = row[0].split(';')[0]
        features['race'] =  row[0].split(';')[1]
        features['numRace'] = -1
        features['Terran'] = 0.0
        features['Protoss'] = 0.0
        features['Zerg'] = 0.0
        if "Terran" in features['race']:
            features['numRace'] = 1.0
            features['Terran'] = 1.0
        if "Protoss" in features['race']:
            features['numRace'] = 2.0
            features['Protoss'] = 1.0
        if "Zerg" in features['race']:
            features['numRace'] = 3.0
            features['Zerg'] = 1.0
        features['s'] = 0
        features['sBase'] = 0
        features['sMineral'] = 0
        features['firstHk'] = ''
        features['firstHkNum'] = -1.0
        features['openingS'] = 0.0
        features['openingSBase'] = 0.0
        features['openingHotkeyBind'] = 0.0
        firstHkFound = False
        features['gameString'] = ''
        features['shortGameString'] = ''
        for i in range(0,10):
            for j in range (0,3):
                features['hotkey%s%s'%(i,j)]= 0
        features['apm'] = 1
        lastFrame = 1
        try:
            op = row[1]
            features['openingMove'] = row[1]
            if "sBase" in op:
                features['openingSBase'] = 1.0
            elif "s" in op:
                features['openingS'] = 1.0
            elif "hotkey" in op:
                features['openingHotkeyBind'] = 1.0
        except:
            #error in data
            pass
        for val in row[1:]:
            if val == "sBase":
                features['sBase'] = features.get("sBase",0) + 1
                features['gameString'] = features['gameString'] + "base"
                features['shortGameString'] = features['shortGameString'] + "b"
            elif val == "s":
                features['s'] = features.get("s",0) + 1
                features['gameString'] = features['gameString'] + "slct"
                features['shortGameString'] = features['shortGameString'] + "s"
            elif val == "sMineral":
                features['sMineral'] = features.get("sMineral",0) + 1
                features['gameString'] = features['gameString'] + "mine"
                features['shortGameString'] = features['shortGameString'] + "m"
            elif 'hotkey' in val:
                features[val] = features.get(val,0) + 1
                features['gameString'] = features['gameString'] + "hk"+val[6:]
                features['shortGameString'] = features['shortGameString'] + "h"
                if not firstHkFound:
                    features['firstHk'] = val
                    features['firstHkNum'] = int(val[6])
                    firstHkFound = True
            else:
                features['apm'] = features.get("apm",0) + 1
                lastFrame = int(val)
            if 's' in val:
                if isInSelectSequence:
                    currentSelectSequence += 1
                else:
                    if currentSelectSequence > 0:
                        selectSequence.append(currentSelectSequence)
                    currentSelectSequence = 1
                    isInSelectSequence = True
                    isInHotkeySequence = False
            if 'hotkey' in val:
                if isInHotkeySequence:
                    currentHotkeySequence += 1
                else:
                    if currentSelectSequence > 0:
                        hotkeySequence.append(currentHotkeySequence)
                    currentHotkeySequence = 1
                    isInSelectSequence = False
                    isInHotkeySequence = True

        features['selectPercent'] = features['s'] / features['apm'] * 100
        features['sBasePercent'] = features['sBase'] / features['apm'] * 100
        if isInSelectSequence:
            selectSequence.append(currentSelectSequence)
        if isInHotkeySequence:
            hotkeySequence.append(currentHotkeySequence)
        total=0
        maxi = 0
        for seq in selectSequence:
            total+=seq
            if maxi < seq:
                maxi = seq
        if len(selectSequence):
            features['averageSelectSequence'] = total / (1.0*len(selectSequence))
        else:
            features['averageSelectSequence'] = 0
        features['maxSelectSequence'] = maxi * 1.0
        total=0
        maxi = 0
        for seq in hotkeySequence:
            total+=seq
            if maxi < seq:
                maxi = seq
        if len(hotkeySequence):                           
            features['averageHotkeySequence'] = total / (1.0*len(hotkeySequence))
        else:
            features['averageHotkeySequence'] = 0   
        features['maxHotkeySequence'] = maxi * 1.0
        if features['firstHk'] == '':
            features['firstHk'] = 'No hotkey used'
        #"""
        features['s'] = features['s'] / features['apm'] * 100
        features['sBase'] = features['sBase'] / features['apm'] * 100
        features['sMineral'] = features['sMineral'] / features['apm'] * 100
        features['nbHotkeysUsed'] = 0
        features['nbHotkeysRealloc'] = 0
        for i in range(0,10):
            features['hotkey%s1'%(i)] = features['hotkey%s1'%(i)] / features['apm'] * 100
            if features['hotkey%s0'%(i)] > 0:
                features['nbHotkeysUsed'] = features['nbHotkeysUsed'] + 1
            if features['hotkey%s2'%(i)] > 0:
                features['nbHotkeysRealloc'] = features['nbHotkeysRealloc'] + 1



        #"""
        features['apm'] = features['apm'] / (lastFrame/1800)#30 fps, shitty computers probably
        #print(features)
        data.append(features)
    return data

In [42]:
all_features = []
all_features = generateData(train_list, "battleneturl")

100%|███████████████████████████████████████████████████████████████████████████████████| 4099/4099 [00:09<00:00, 425.36it/s]


In [43]:
test_features = []
test_features = generateData(test_list, "row ID")

100%|███████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:02<00:00, 437.23it/s]


In [44]:
with open('features.csv', 'w') as outfile:
    fp = csv.DictWriter(outfile, all_features[0].keys())
    fp.writeheader()
    fp.writerows(all_features)

In [45]:
with open('test_features.csv', 'w') as outfile:
    fp = csv.DictWriter(outfile, test_features[0].keys())
    fp.writeheader()
    fp.writerows(test_features)

In [30]:
1/1600

0.000625

In [18]:
all_features[1]

OrderedDict([('battleneturl', '/MMA/'),
             ('race', 'Terran'),
             ('numRace', 1.0),
             ('Terran', 1.0),
             ('Protoss', 0.0),
             ('Zerg', 0.0),
             ('s', 22.61072261072261),
             ('sBase', 0.0),
             ('sMineral', 0.3108003108003108),
             ('firstHk', 'hotkey10'),
             ('firstHkNum', 1),
             ('openingS', 1.0),
             ('openingSBase', 0.0),
             ('openingHotkeyBind', 0.0),
             ('gameString',
              'slctslctslcthk10hk20hk30hk40hk50hk60hk00hk70hk90hk80slcthk22slcthk22slcthk22slctslcthk22slcthk22slcthk22slcthk40hk22hk42hk22hk42hk22hk42hk22hk42slcthk42slcthk42hk22slctslcthk22hk42hk22slcthk22slcthk22hk42hk22slcthk22slcthk22hk42hk22slcthk22slcthk22slcthk22hk42hk22hk42slcthk22hk42slcthk42slctslcthk22hk42hk22slcthk40hk22slcthk42hk22hk42hk22slcthk22slcthk22hk42hk22hk42hk22slcthk22slcthk22hk42hk22hk42hk22slcthk22hk42hk22slcthk22slcthk22hk42hk22slcthk22hk42hk22slcthk22sl